In [1]:
from gates import *

In [2]:
def Nor(a, b):
    return Not(Or(a, b))

In [53]:
class SRNorLatch:
    def __init__(self):
        self.q = 0
        self.not_q = 1

    def eval(self, r, s):
        if s == 1:
            self.not_q = Nor(self.q, s)
            self.q = Nor(r, self.not_q)
        elif r == 1:
            self.q = Nor(r, self.not_q)
            self.not_q = Nor(self.q, s)

        return self
    
    def __repr__(self):
        return f'q={self.q}, not_q={self.not_q}'

In [54]:
l = SRNorLatch()
l

q=0, not_q=1

In [59]:
l.eval(s=1, r=0)

q=1, not_q=0

In [62]:
l.eval(s=0, r=1)

q=0, not_q=1